## This notebook is to ingest the rz1, rz2, rz3, rz4, busi_def, glossary into cloudant database

**Remember:**   
the mapping /result in docker image is mapped to local folder `~/Documents/GitHub/jupyter/result`  
the mapping /input in docker image is mapped to local folder `~/Documents/GitHub/jupyter/input`  


link to the cloudant database is here:   
https://415ab4c2-a5d1-4acc-bd9a-6ffc2a7ff356-bluemix.cloudant.com/dashboard.html    

after the data is ingested, use the following api to refresh the odmmeta api server   
https://odmmeta.dal1a.ciocloud.nonprod.intranet.ibm.com/

use the following api to test the odmmeta server  
https://odmmeta.dal1a.ciocloud.nonprod.intranet.ibm.com/igc/get_term/RCNUM

**assumption of this notebook:**  
1. the schema for odm is ODMPRD and the schema of ODMR is IDMPRD (the IDMDB is not included yet)
1. For those columns which can not be found in sysibm.syscolumns, it will be discarded.
1. For those table which can not be found in sysibmn.systables, it will be discarded. 
1. use the defualt cloudant user id and password. 
1. we put all RZ1, RZ2, RZ3, RZ4 contents in the same cloudant database.
1. the key for RZ1 and RZ3 is the table name (not TID). the key for RZ2 and RZ4 is the table name + column name. (not tid + column name)



In [ ]:
import sys
import time
sys.path.append('/odm_modules')
from common_func import cloudant_conn as cc
from common_func import odm_conn
import pandas as pd
import re
import os
import docx2txt
cc.cloudant_client.connect()

In [ ]:
#environ_suffix = '_test'  # it should be either _test or '' 
environ_suffix = ''  # it should be either _test or '' 

In [ ]:
converter_rz2 = { 
    'CTID': 'table_id',
    'ALTER_ID': 'alter_id',
    'CTABNAME': 'table_name',
    'CCOLNAME': 'column_name',
    'CCOLFMT': 'format',
    'TCOLNAME': 'description',
    'CCLASS': 'data_class',
#    'TEXTRA': 'extra information',
    'TBCREATOR': 'schema',
    'FORMAT': 'format',
    'KEYSEQ': 'key_sequence',
    'COLNO': 'column_number'
    }
converter_rz1 = {'CTABNAME': 'table_name', 'ALTER_ID': 'alter_id','TTID': "description", 'CTID':'table_id'}

## ingest RZ1 table

In [ ]:
cloudant_db_name = 'rz1{}'.format(environ_suffix) # the cloudant database name to be ingested
query_rz1 = """
select CTID, CTABNAME, TTID, CTABNAME as ALTER_ID
from odmprd.odmt_ddict_tables   
where 
1 = 1
--AND CTID = 'E02'
"""

res =  cc.odm_2_cloudant(query_rz1, 'prod', cloudant_db_name, 
                         converter = converter_rz1, 
                         keys = ['CTID'] , 
                         mode = 'REPLACE', 
                         src_code = 'rz1',
                         drop_origin_keys = True)
print(res)

# create dataframe for odmmeta
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(query_rz1)
    df_rz1 = pd.DataFrame(result).rename(columns = converter_rz1)
df_rz1.head()

## Ingest RZ3 table

In [ ]:
cloudant_db_name = 'rz3{}'.format(environ_suffix) # the cloudant database name to be ingested
query_rz3 = """
select CTID, CTABNAME, TTID , CTABNAME as ALTER_ID
from odmprd.ODMT_DDICT_TBLODMR   
where 
1 = 1
--AND CTID = 'E02'
"""
res =  cc.odm_2_cloudant(query_rz3, 'prod', cloudant_db_name, 
                         converter = converter_rz1, 
                         keys = ['CTID'] , 
                         mode = 'REPLACE', 
                         src_code = 'rz3')
print(res)

# create dataframe for odmmeta
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(query_rz3)
    df_rz3 = pd.DataFrame(result).rename(columns = converter_rz1)
df_rz3.head()

## Ingest RZ2 table

In [ ]:
ctid_list = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
gp_cnt = 4
gps = [ctid_list[start::gp_cnt] for start in range(gp_cnt)]
gps_cond = [ ', '.join(  ["'{}'".format(ch)      for ch in gp ]) for gp in gps]
gps_cond

In [ ]:
cloudant_db_name = 'rz2{}'.format(environ_suffix)
cc.cloudant_client.delete_database(cloudant_db_name) # remove the original contents todo full load-replacement
for gp_cond in gps_cond: 
    query_rz2 = """
    select RZ2.CTID, 
    RZ2.CCOLNAME, 
    RZ2.TCOLNAME,
    TRIM(RZ1.CTABNAME) || '.'||TRIM(RZ2.CCOLNAME) AS ALTER_ID, 
    RZ2.CCLASS,  
    --RZ2.TEXTRA, 
    RZ1.CTABNAME, 
    SYC.TBCREATOR, 
    SYC.COLNO, 
    SYC.COLTYPE|| '('||SYC.LENGTH||')' AS FORMAT,
    SYC.KEYSEQ
    from 
    odmprd.odmt_ddict_columns RZ2, 
    ODMPRD.odmt_ddict_tables RZ1,
    sysibm.syscolumns SYC
    where 
    RZ2.fdiscont <> 'Y' 
    and RZ2.CLANGUAG = '' 
    AND RZ1.CTID = RZ2.CTID 
    and SYC.TBNAME = RZ1.CTABNAME
    and SYC.NAME = RZ2.CCOLNAME
    AND SYC.TBCREATOR = 'ODMPRD'
    AND SUBSTR(RZ1.CTID,1,1) IN ({})
    --AND RZ1.CTID = 'E02'
    """.format(gp_cond)

    res = cc.odm_2_cloudant(query_rz2, 'prod', cloudant_db_name,  
                         converter = converter_rz2, 
                         keys = ['CTID', 'CCOLNAME'] , 
                         mode = 'APPEND',     # Must be append for RZ2 and RZ4
                         src_code = 'rz2',
                         drop_origin_keys = False)
    print(res)
    

In [ ]:
query_rz2 = """
select RZ2.CTID, 
RZ2.CCOLNAME, 
RZ2.TCOLNAME,
TRIM(RZ1.CTABNAME) || '.'||TRIM(RZ2.CCOLNAME) AS ALTER_ID, 
RZ2.CCLASS,  
--RZ2.TEXTRA, 
RZ1.CTABNAME, 
SYC.TBCREATOR, 
SYC.COLNO, 
SYC.COLTYPE|| '('||SYC.LENGTH||')' AS FORMAT,
SYC.KEYSEQ
from 
odmprd.odmt_ddict_columns RZ2, 
ODMPRD.odmt_ddict_tables RZ1,
sysibm.syscolumns SYC
where 
RZ2.fdiscont <> 'Y' 
and RZ2.CLANGUAG = '' 
AND RZ1.CTID = RZ2.CTID 
and SYC.TBNAME = RZ1.CTABNAME
and SYC.NAME = RZ2.CCOLNAME
AND SYC.TBCREATOR = 'ODMPRD'
"""

# create dataframe for odmmeta
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(query_rz2)
    df_rz2 = pd.DataFrame(result).rename(columns = converter_rz2)
df_rz2.head()
    

In [ ]:
df_rz2.shape

## Ingest RZ4 table

In [ ]:
ctid_list = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
gp_cnt = 8
gps = [ctid_list[start::gp_cnt] for start in range(gp_cnt)]
gps_cond = [ ', '.join(  ["'{}'".format(ch)      for ch in gp ]) for gp in gps]
gps_cond

In [ ]:
cloudant_db_name = 'rz4{}'.format(environ_suffix)
cc.cloudant_client.delete_database(cloudant_db_name) # full load replacement
for gp_cond in gps_cond: 
    query_rz4 = """
    select RZ4.CTID, 
    RZ4.CCOLNAME, 
    RZ4.TCOLNAME, 
    TRIM(RZ3.CTABNAME) || '.'||TRIM(RZ4.CCOLNAME) AS ALTER_ID, 
    RZ4.CCLASS,  
    --RZ4.TEXTRA, 
    RZ3.CTABNAME, 
    SYC.TBCREATOR, 
    SYC.COLNO, 
    SYC.COLTYPE|| '('||SYC.LENGTH||')' AS FORMAT,
    SYC.KEYSEQ
    from 
    odmprd.ODMT_DDICT_COLODMR RZ4, 
    ODMPRD.ODMT_DDICT_TBLODMR RZ3,
    sysibm.syscolumns SYC
    where 
    RZ4.fdiscont <> 'Y' 
    and RZ4.CLANGUAG = '' 
    AND RZ3.CTID = RZ4.CTID 
    and SYC.TBNAME = RZ3.CTABNAME
    and SYC.NAME = RZ4.CCOLNAME
    AND SYC.TBCREATOR = 'IDMPRD'
    AND SUBSTR(RZ3.CTID, 1,1) IN ({})
    --AND RZ3.CTID = 'E02'
    """.format(gp_cond)
    res = cc.odm_2_cloudant(query_rz4, 'prod', cloudant_db_name,  
                         converter = converter_rz2, 
                         keys = ['CTID', 'CCOLNAME'] , 
                         mode = 'APPEND', 
                         src_code = 'rz4',
                         drop_origin_keys = False)
    print(res)

In [ ]:
# for odmmeta database
query_rz4 = """
select RZ4.CTID, 
RZ4.CCOLNAME, 
RZ4.TCOLNAME, 
TRIM(RZ3.CTABNAME) || '.'||TRIM(RZ4.CCOLNAME) AS ALTER_ID, 
RZ4.CCLASS,  
--RZ4.TEXTRA, 
RZ3.CTABNAME, 
SYC.TBCREATOR, 
SYC.COLNO, 
SYC.COLTYPE|| '('||SYC.LENGTH||')' AS FORMAT,
SYC.KEYSEQ
from 
odmprd.ODMT_DDICT_COLODMR RZ4, 
ODMPRD.ODMT_DDICT_TBLODMR RZ3,
sysibm.syscolumns SYC
where 
RZ4.fdiscont <> 'Y' 
and RZ4.CLANGUAG = '' 
AND RZ3.CTID = RZ4.CTID 
and SYC.TBNAME = RZ3.CTABNAME
and SYC.NAME = RZ4.CCOLNAME
AND SYC.TBCREATOR = 'IDMPRD'
"""
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(query_rz4)
    df_rz4 = pd.DataFrame(result).rename(columns = converter_rz2)
df_rz4.shape

## Ingest Business Definition Table   


#### Now ingest the business definition document  

the business definition document is in the following link:   
https://apps.na.collabserv.com/communities/service/html/communityview?communityUuid=84d3cd10-4160-48e1-abb6-f65af679ee28#fullpageWidgetId=Wabd63ea0f599_4dc3_aed5_9ffce68576e0&file=ca75dc47-c7f3-4fde-8249-3e2bc7b05a9b

**move the business definition document into the following folder:**  
~/Documents/GitHub/jupyter/input/Business_Definitions.docx

``` 
rm ~/Documents/GitHub/jupyter/input/Business_Definitions.docx
mv ~/Downloads/"ODM Business Definitions.docx"  ~/Documents/GitHub/jupyter/input/Business_Definitions.docx 
```

In [ ]:
cloudant_db_name = 'busi_def{}'.format(environ_suffix)
busi_file = '/input/Business_Definitions.docx'
text = docx2txt.process(busi_file)

In [ ]:
# f_strip to remove the leading blank and tailing blank , blank lines. 
def f_strip(x):
    y = {}
    for item in x: 
        y[item] = x[item].strip().replace('\n\n', '\n')
    return y
# remove the head and tail,  4 # is the mark added manually in the document and it is hidden in the file with white color 
p = re.compile(r'.*#{4}(.*)End\sof\sDocument', re.DOTALL )
newText= p.match(text).groups()[0]
p = re.compile(r'\n\s+') 
newText = p.sub('\n', newText)
p = re.compile(r'\n+') 
newText = p.sub('\n', newText)
# replace valid values: to be Allowed Values:
p = re.compile(r'valid values\s*:', re.I)
newText = p.sub('Allowed Values:', newText)
# use findall to split the text by business terms
x=re.compile(r'.*?Business\s+definition\:.*?Format\:.*?Allowed\s+Values\:.*?(?=\n[^\n]*\nBusiness\s+definition\:)', re.S|re.I)
lst = x.findall(newText+ '\n \nBusiness definition:') # attach str 'business definition' so that the last business term can be matched
# remove the blank lines 
p = re.compile('^$\n', re.DOTALL|re.MULTILINE)
newLst = list(map(lambda y: p.sub('', y), lst))
#parse every business term block to get term name, definition, format, and allowed values
p = re.compile(r'(.*)Business\s+definition\:(.*?)Format\:(.*?)Allowed\s+Values\:(.*)', re.S|re.I) 
newLst = list(map(lambda x: p.match(x).groups(), newLst))
# get the short description and term name from the first element in each item
# the parsed item will be 'short desc', 'term name', 'tail'. the short desc + tail will become final short desc
p = re.compile(r'(.*)\(\s*(.*)\s*\)(.*)')
def f(x): # f(x) is to parse the title line of every business term to get the short description and the term name
    y=p.match(x[0])
    if y: 
        z = y.groups() + x[1:]
    else: 
        print('\n' )
        print(str( x))
        z = ('NA', 'NA')+ x[1:]
    return z
newLst = list(map(f, newLst))

fields = [ 'Term Name','Short Description', 'Long Description', 'Data Type', 'Allowed Values']
busiTermDictList = list(map(lambda x: dict(zip(fields,(x[1], x[0]+x[2]) + x[3:])), newLst))
busiTermDictList = list(map(f_strip, busiTermDictList))

In [ ]:
busi_df = pd.DataFrame(busiTermDictList)
converter_busi = {'Long Description': 'long_description', 
                  'Short Description': 'description', 
                  'Data Type': 'format', 
                 'Allowed Values': 'allowed_values'}
busi_df = busi_df.loc[busi_df["Term Name"] !='']
# get the duplicate term in the list
busi_df.set_index('Term Name').loc[busi_df.groupby('Term Name')['Short Description'].count()==2].sort_index()

In [ ]:
busiTermDictList_new = [{'_'.join(map(lambda s: s.upper(), k.split())):v   for k, v in element.items()}for element in busiTermDictList]
busiTermDictList_new

In [ ]:
import json

for element in busiTermDictList_new: 
    print(element['TERM_NAME'].replace('/', '_'))
    with open('/result/busi_df/{}.json'.format(element['TERM_NAME']).replace('/', '_'), 'w') as f: 
        json.dump(element, f)
        

In [ ]:
busi_df.to_excel('busi_def.xlsx', index= False)

In [ ]:
cc.df_2_cloudant(busi_df, cloudant_db_name, 
                 keys = ['Term Name'], 
                 converter = converter_busi, 
                 mode = 'REPLACE', 
                src_code = 'busi_def',
                drop_origin_keys = True)


In [ ]:
# tbname is the df using tbname as key to be ingested in cloudant db
df_rz13_tbname = pd.concat([df_rz1, df_rz3]).applymap(lambda x: x.strip()).drop(columns = ['alter_id']) 
def process_df13(df):
    return df.apply(lambda col: ', '.join(col.drop_duplicates()))
df_rz13_tid = df_rz13_tbname.groupby('table_id').apply(process_df13).drop(columns = ['table_id']).reset_index()
#df_rz13_tid.loc[df_rz13_tid.table_id == 'D01']
#df_rz13_tbname.loc[df_rz13_tbname.table_id == 'E02']


In [ ]:
#cols = ['table_id', 'column_name', 'description', 'data_class','table_name','format', 'schema']
lst = list(busi_df)
busi_df[lst] = busi_df[lst].astype(str)
busi_df = busi_df.applymap(lambda x: x.strip())
cols = [ 'column_name', 'description', 'data_class','table_name','format']

df_rz24 = pd.concat([df_rz2, df_rz4])
df_rz24[list(df_rz24)] = df_rz24[list(df_rz24)].astype(str).applymap(lambda x: x.strip())
df_rz24.table_name = + df_rz24.table_id + ' ' + df_rz24.schema + '.' + df_rz24.table_name
df_rz24 = df_rz24.reindex(columns = cols)
#df_rz24_test = df_rz24.loc[df_rz24.column_name == 'CFORMSTA']
df_rz24 = df_rz24.merge(busi_df, left_on = 'column_name', right_on = 'Term Name', how = 'left').drop(columns = ['Term Name', 'Short Description', 'Data Type']).fillna('')


def process_df_rz24(df):
    ser = pd.Series()
    for col in df.columns:
        if col == 'table_name':
            ser[col] = ', '.join(df[col].drop_duplicates())
        else:
            ser[col] = df[col].value_counts().idxmax()
    return ser
df_rz24_new = df_rz24.groupby('column_name').apply(process_df_rz24)
df_rz24_new = df_rz24_new.drop(columns = ['column_name']).reset_index()
df_rz24_new.rename(columns = {'table_name': 'where'}, inplace = True)
            


In [ ]:
#print('\n'.join(df_rz24_new.loc[(df_rz24_new.column_name == 'CSALBIW'), ['table_name']].table_name.values[0].split(',') ))
# df_rz24.loc[(df_rz24.column_name == 'CSALBIW')]['table_name']

In [ ]:
#df_rz24_new.loc[df_rz24_new.column_name == 'RCNUM'].T

#df_rz24_new = df_rz24_new.merge(busi_df, left_on = 'column_name', right_on = 'Term Name', how = 'left').drop(columns = ['Term Name', 'Short Description', 'Data Type'])
# there are duplicated in busi_df, just run the follow code to remove the duplicates
#df_rz24_new = df_rz24_new.groupby('column_name').apply(process_df_rz24)
#df_rz24_new = df_rz24_new.drop(columns = ['column_name']).reset_index()



## Ingest Glossary Table  


#### now load the glos document

the glossary document is in the following link: 
https://apps.na.collabserv.com/communities/service/html/communityview?communityUuid=84d3cd10-4160-48e1-abb6-f65af679ee28#fullpageWidgetId=Wabd63ea0f599_4dc3_aed5_9ffce68576e0&file=c9acb4a9-a5e8-43d4-8687-1e2bfcf4ce9d

use the following commands to move it the working folder:
```
rm ~/Documents/GitHub/jupyter/input/Glossary_1.xlsx
mv ~/Downloads/"Glossary of Acronyms 2019-06-27.xlsx"  ~/Documents/GitHub/jupyter/input/Glossary_1.xlsx 
```

In [ ]:
cloudant_db_name = 'glossary{}'.format(environ_suffix)
glos_file = '/input/Glossary_1.xlsx'
df_glos = pd.read_excel(glos_file).fillna('')
df_glos = df_glos.rename(index=str, columns = {"Acronym/Term": "Term Name"}) 
df_glos = df_glos.loc[~df_glos['Term Name'].isin(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')) ]
df_glos['description'] = df_glos.Explanation + '\n' + df_glos.Description
df_glos.drop(columns = {'Explanation', 'Description'}, inplace = True)
cc.df_2_cloudant(df_glos, cloudant_db_name, 
                 keys = ['Term Name'], 
                 mode = 'REPLACE', 
                src_code = 'glossary')




### Now let's populate the odmmeta database.
we have 
+ df_rz13_tbname
+ df_rz13.tid
+ df_rz24_new
+ df_glos

In [ ]:
df_rz13_tbname['category'] = 'table'
df_rz13_tid['category'] = 'table'
df_glos['category'] = 'glossory'
df_rz24_new['category'] = 'column'

In [ ]:
df_rz13_tbname.rename(columns = {'table_name': '_id'}, inplace = True)
df_rz13_tid.rename(columns = {'table_id': '_id'}, inplace = True)
df_glos.rename(columns = {'Term Name': '_id'}, inplace = True)
rz13_tbname_dict = df_rz13_tbname.to_dict(orient = 'record')
rz13_tid_dict = df_rz13_tid.to_dict(orient = 'record')
glos_dict = df_glos.to_dict(orient = 'record')


In [ ]:
df_rz24_new = df_rz24_new.rename(columns = {'column_name': '_id'})
rz24_rows = df_rz24_new.fillna('').to_dict(orient = 'record')
rz24_rows_dict = [ dict(filter(lambda x: x[1] != '' , row.items())) for row in rz24_rows]
rz24_rows_dict

In [ ]:
df_rz24_new.loc[df_rz24_new._id == 'CFORMSTA'].T

In [ ]:
cloud_db_name = 'odmmeta'
cc.cloudant_client.connect()
cc.cloudant_client.delete_database(cloud_db_name)
my_database = cc.cloudant_client.create_database(cloud_db_name) 

In [ ]:
my_database.bulk_docs(rz13_tbname_dict)
my_database.bulk_docs(rz13_tid_dict)
my_database.bulk_docs(rz24_rows_dict)
my_database.bulk_docs(glos_dict)

# INGESTION STOPS HERE


## Access cloudant database via python APIs